In [ ]:
#| default_exp datasources.aeronautica
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases aeronáuticas fazendo novas requisições às APIS

In [ ]:
#| export
import os
from typing import List, Union
from functools import cached_property


import numpy as np
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.xtras import Path
from fastcore.parallel import parallel

from extracao.datasources.icao import get_icao
from extracao.datasources.aisweb import get_aisw
from extracao.datasources.aisgeo import get_aisg
from extracao.datasources.redemet import get_redemet
from extracao.datasources.base import Base
from extracao.format import merge_on_frequency

In [ ]:
#| export
load_dotenv(find_dotenv())


True

In [ ]:
__file__ = Path.cwd().parent / "extracao" / "datasources" / "datasources.py"

In [ ]:
#|export
class Aero(Base):
	"""Classe auxiliar para agregar os dados das APIs aeronáuticas"""

	@property
	def stem(self):
		return 'aero'

	@property
	def columns(self):
		return ['Frequência', 'Latitude', 'Longitude', 'Entidade', 'Fonte']

	@cached_property
	def extraction(self) -> pd.DataFrame:
		func = lambda f: f()
		radares = pd.read_csv(Path(__file__).parent / 'arquivos' / 'radares.csv')
		radares['Fonte'] = 'RADARES'
		sources = [get_icao, get_aisw, get_aisg, get_redemet]
		dfs = parallel(func, sources, threadpool=True, progress=True)
		dfs.append(radares)
		return dfs

	def _format(
		self,
		dfs: List,  # List with the individual API sources
	) -> pd.DataFrame:  # Processed DataFrame
		if dfs:
			icao = dfs.pop(0)
			for df in dfs:
				icao = merge_on_frequency(icao, df)

			icao = icao.sort_values(by=icao.columns.to_list(), ignore_index=True)
			icao = icao.drop_duplicates(
				subset=['Frequency', 'Latitude', 'Longitude'],
				keep='last',
				ignore_index=True,
			)
			icao = icao.astype(
				{
					'Frequency': 'float64',
					'Latitude': 'float32',
					'Longitude': 'float32',
					'Description': 'string',
				}
			)
			icao.loc[np.isclose(icao.Longitude, -472.033447), 'Longitude'] = -47.2033447
			icao.loc[np.isclose(icao.Longitude, 69.934998), 'Longitude'] = -69.934998
			return icao.rename(columns={'Frequency': 'Frequência', 'Description': 'Entidade'})


In [ ]:
#| export
# | export# | export
# | export

==================================================ICAO==================================================


,Frequency,Latitude,Longitude,Description
0,113.4,-9.866666666666667,-56.1,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.25,-52.25,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
4,112.0,-10.983333333333333,-37.06666666666667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2399,1176,-20.766666666666666,-51.55,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2400,1082.0,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2401,1019,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2402,1126.0,-20.25,-40.28333333333333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"


==================================================AISWEB==================================================


In [ ]:
a = Aero()
a.update()

,Frequency,Latitude,Longitude,Description
0,2800,-16.201531,-40.674153,[RMET] Radar - Almenara/MG
1,2800,-1.406667,-48.461389,[RMET] Radar - Belém/PA
2,2800,2.844166667,-60.70027777778,[RMET] Radar - Boa Vista/RR
3,2800,-31.404,-52.701644,[RMET] Radar - Canguçu/RS
4,2800,-7.595833,-72.767778,[RMET] Radar - Cruzeiro do Sul/AC
5,2800,-15.97643,-48.016142,[RMET] Radar - Gama/DF
6,2800,-20.27855,-54.47396,[RMET] Radar - Jaraguari/MS
7,2800,-3.149216,-59.991881,[RMET] Radar - Manaus/AM
8,2800,-0.047222,-51.097778,[RMET] Radar - Macapá/AP
9,2800,-9.55129,-35.77068,[RMET] Radar - Maceió/AL


In [ ]:
from extracao.format import merge_on_frequency

In [ ]:
radares = pd.read_csv(Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'radares.csv')
df_new = icao.copy()
for df in [aisw, aisg, redemet, radares]:
   df_new = merge_on_frequency(df_new, df)

In [ ]:
display(icao[icao.Description.str.contains('[ICAO]', regex=False)])


,Frequency,Latitude,Longitude,Description
0,113.4,-9.866666666666667,-56.1,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.25,-52.25,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
4,112.0,-10.983333333333333,-37.06666666666667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2177,130.3,-14.86361111111111,-40.863055555555555,"[ICAO] AOC U 100/100, VITÓRIA DA CONQUISTA"
2178,131.875,-14.86361111111111,-40.863055555555555,"[ICAO] AOC U 100/100, VITÓRIA DA CONQUISTA"
2179,132.0,-14.86361111111111,-40.863055555555555,"[ICAO] AOC U 100/100, VITÓRIA DA CONQUISTA"
2180,121.5,-14.86361111111111,-40.863055555555555,"[ICAO] EMERG 261/450, VITÓRIA DA CONQUISTA"


In [ ]:
display(icao[icao.Description.str.contains('[DOC]', regex=False)])


,Frequency,Latitude,Longitude,Description
2182,1105.0,-9.866666666666667,-56.1,"[DOC] VOR/DME, ALTA FLORESTA (Airbone DME)"
2183,1168,-9.866666666666667,-56.1,"[DOC] VOR/DME, ALTA FLORESTA (Ground-based DME)"
2184,1103.0,-3.25,-52.25,"[DOC] VOR/DME, ALTAMIRA (Airbone DME)"
2185,1166,-3.25,-52.25,"[DOC] VOR/DME, ALTAMIRA (Ground-based DME)"
2186,1146.0,-4.183333333333334,-69.93333333333334,"[DOC] VOR/DME, AMAZONICA (Airbone DME)"
...,...,...,...,...
2399,1176,-20.766666666666666,-51.55,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2400,1082.0,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2401,1019,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2402,1126.0,-20.25,-40.28333333333333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"


In [ ]:
df_new[df_new.Description.str.contains('ICAO')]

,Frequency,Latitude,Longitude,Description
0,115.7,-20.466666666666665,-54.666666666666664,"[ICAO] VOR/DME, CAMPO GRANDE INTL."
1,116.0,-17.65,-39.25,"[ICAO] VOR, CARAVELAS"
2,116.0,-23.4,-46.38333333333333,"[ICAO] VOR/DME, SAO PAULO GUARULHOS INTL."
3,115.5,-27.683333333333334,-48.5,"[ICAO] VOR/DME, FLORIANOPOLIS HERCILIO LUZ"
4,115.5,-20.25,-40.28333333333333,"[ICAO] VOR/DME, VITORIA GOIABEIRAS"
...,...,...,...,...
2580,116.9,-8.133333333333333,-34.93333333333333,"[ICAO] VOR/DME, RECIFE GUARARAPES | [AISG] VOR..."
2581,117.0,-19.683333333333334,-47.06666666666667,"[ICAO] VOR, ARAXA | [AISG] VOR - ARAXA"
2582,117.3,-1.3833333333333333,-48.483333333333334,"[ICAO] VOR/DME, BELEM VAL DE CAES | [AISG] VOR..."
2583,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA | [AISG] VOR - LETICIA"


In [ ]:
df_new[df_new.Description.str.contains('[AISW]', regex=False)]

,Frequency,Latitude,Longitude,Description
1636,109.1,-25.6,-54.46666666666667,"[ICAO] ILS, FOZ DO IGUACU CATARATAS | [AISW] S..."
1637,109.1,-7.279722222222222,-35.89277777777778,"[AISW] SBKG-NAV, LOC/DME 15 IKG, Presidente Jo..."
1642,109.3,-1.3666666666666667,-48.46666666666667,"[ICAO] ILS/DME, BELEM VAL DE CAES | [AISW] SBB..."
1643,109.3,-22.8,-43.21666666666667,"[ICAO] ILS, RIO DE JANEIRO INTL. | [AISW] SBGL..."
1644,109.3,-23.633333333333333,-46.65,"[ICAO] ILS, SAO PAULO CONGONHAS | [AISW] SBSP-..."
...,...,...,...,...
2495,75.0,-23.43611111111111,-46.488055555555555,"[AISW] SBGR-NAV, IM 10L IUC, Guarulhos - Gover..."
2496,122.65,-22.791666666667,-45.204444444444,"[AISW] SBGW-COM, Operações, Campo Edu Chaves"
2497,1093.0,-15.872777777777777,-47.93833333333333,"[AISW] SBBR-NAV, ILS/DME 29R IND, Presidente J..."
2498,135.1,-22.81,-43.250555555556,"[AISW] SBGL-COM, Tráfego, Galeão - Antônio Car..."


In [ ]:
aisw[aisw.Description.str.contains('[AISW]', regex=False)]

,Frequency,Latitude,Longitude,Description
0,118.7,-5.908333333333,-35.249166666667,"[AISW] SBNT-COM, Torre, CAMPO AUGUSTO SEVERO"
1,122.8,-5.908333333333,-35.249166666667,"[AISW] SBNT-COM, Torre, CAMPO AUGUSTO SEVERO"
2,121.9,-5.908333333333,-35.249166666667,"[AISW] SBNT-COM, Solo, CAMPO AUGUSTO SEVERO"
3,132.65,-5.908333333333,-35.249166666667,"[AISW] SBNT-COM, ATIS, CAMPO AUGUSTO SEVERO"
4,109.3,-5.924722222222223,-35.23722222222222,"[AISW] SBNT-NAV, ILS/DME 16L INT, CAMPO AUGUST..."
...,...,...,...,...
359,130.675,-22.81,-43.250555555556,"[AISW] SBGL-COM, Pátio, Galeão - Antônio Carlo..."
360,131.05,-22.81,-43.250555555556,"[AISW] SBGL-COM, Pátio, Galeão - Antônio Carlo..."
361,110.5,-22.839166666666667,-43.239444444444445,"[AISW] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antô..."
362,109.3,-22.796666666666667,-43.223888888888894,"[AISW] SBGL-NAV, ILS 10 CAT II, Galeão - Antôn..."


In [ ]:
aisw[aisw.Description.str.contains('[DOC]', regex=False)]

,Frequency,Latitude,Longitude,Description
364,332.0,-5.924722222222223,-35.23722222222222,"[DOC] SBNT-NAV, ILS/DME 16L INT, CAMPO AUGUSTO..."
365,1054.0,-5.924722222222223,-35.23722222222222,"[DOC] SBNT-NAV, ILS/DME 16L INT, CAMPO AUGUSTO..."
366,991,-5.924722222222223,-35.23722222222222,"[DOC] SBNT-NAV, ILS/DME 16L INT, CAMPO AUGUSTO..."
367,332.0,-3.1502777777777777,-59.98222222222223,"[DOC] SBMN-NAV, ILS/DME 09 IPE, Campo Ponta Pe..."
368,1054.0,-3.1502777777777777,-59.98222222222223,"[DOC] SBMN-NAV, ILS/DME 09 IPE, Campo Ponta Pe..."
...,...,...,...,...
485,1054.0,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
486,991,-22.796666666666667,-43.223888888888894,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."
487,332.9,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."
488,1076.0,-22.804444444444446,-43.26416666666667,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn..."


In [ ]:
df_new[df_new.Description.str.contains('[AISG]', regex=False)]

,Frequency,Latitude,Longitude,Description
2539,112.0,-29.716666666666665,-53.71666666666667,"[ICAO] VOR/DME, SANTA MARIA | [AISG] VOR - SAN..."
2540,112.0,-7.6,-72.76666666666667,"[ICAO] VOR/DME, CRUZEIRO DO SUL INTL. | [AISG]..."
2541,112.1,-12.7,-60.083333333333336,"[ICAO] VOR/DME, VILHENA | [AISG] VOR - VILHENA"
2542,112.1,-25.583333333333332,-54.5,"[ICAO] VOR/DME, FOZ DO IGUACU CATARATAS | [AIS..."
2543,112.1,-9.366666666666667,-40.56666666666667,"[ICAO] VOR/DME, PETROLINA | [AISG] VOR - PETRO..."
...,...,...,...,...
2810,115.9,-5.771811111,-35.368680556,[AISG] VOR - SAO GONCALO DO AMARANTE
2811,116.1,-27.13569286,-52.66296794,[AISG] VOR - CHAPECO
2812,116.7,-2.371485,-44.397097222,[AISG] VOR - ALCANTARA
2813,117.5,-15.865012778,-47.900188889,[AISG] VOR - KUBITSCHEK


In [ ]:
aisg[aisg.Description.str.contains('[AISG]', regex=False)]

,Frequency,Latitude,Longitude,Description
0,0.3,-25.402615833,-49.228916667,[AISG] NDB - BACACHERI NDB NOT AVBL BEYOND 50 NM
1,0.34,-22.168888889,-49.069722222,[AISG] NDB - AREALVA
2,0.23,-7.266,-35.892666667,[AISG] NDB - CAMPINA GRANDE NDB NOT AVBL BEYON...
3,0.31,-21.986061111,-47.337497222,[AISG] NDB - PIRASSUNUNGA
4,0.34,3.860173917,-51.800133944,[AISG] NDB - OIAPOQUE NDB NOT AVBL BEYOND 70 NM
...,...,...,...,...
271,1191.0,-15.979906306,-48.032267167,[AISG] DME - GAMA 104X
272,1176.0,-9.3988875,-38.245475,[AISG] DME - PAULO AFONSO 89X
273,1166.0,-10.874568056,-61.846398889,[AISG] DME - JI-PARANÃ 79X
274,1188.0,-10.98192556,-37.077322222,[AISG] DME - CAJU 101X


In [ ]:
aisg[aisg.Description.str.contains('[DOC]', regex=False)]

,Frequency,Latitude,Longitude,Description


In [ ]:
df_new[df_new.Description.str.contains('[RMET]', regex=False)]

,Frequency,Latitude,Longitude,Description
2815,2800,-16.201531,-40.674153,[RMET] Radar - Almenara/MG
2816,2800,-1.406667,-48.461389,[RMET] Radar - Belém/PA
2817,2800,2.844166667,-60.70027777778,[RMET] Radar - Boa Vista/RR
2818,2800,-31.404,-52.701644,[RMET] Radar - Canguçu/RS
2819,2800,-7.595833,-72.767778,[RMET] Radar - Cruzeiro do Sul/AC
2820,2800,-15.97643,-48.016142,[RMET] Radar - Gama/DF
2821,2800,-20.27855,-54.47396,[RMET] Radar - Jaraguari/MS
2822,2800,-3.149216,-59.991881,[RMET] Radar - Manaus/AM
2823,2800,-0.047222,-51.097778,[RMET] Radar - Macapá/AP
2824,2800,-9.55129,-35.77068,[RMET] Radar - Maceió/AL


In [ ]:
df_new[df_new.Description.str.contains('.*\|.*\|', regex=True)].shape

(0, 4)

In [ ]:
df_new[df_new.Description.str.contains('\[DOC\].*\[DOC\]', regex=True)].shape

(48, 4)

In [ ]:
df_new[df_new.Description.str.contains('[DOC]', regex=False)].shape

(454, 4)

In [ ]:
222+126+172

520

In [ ]:
454 + 48

502

In [ ]:
def update_aero(
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a base de dados de emissões da aeronáutica"""
    icao = get_icao()
    aisw = get_aisw()
    aisg = get_aisg()
    redemet = get_redemet()
    radares = pd.read_csv(os.environ["PATH_RADAR"])
    for df in [aisw, aisg, redemet, radares]:
        icao = merge_on_frequency(icao, df)

    icao = icao.sort_values(by=icao.columns.to_list(), ignore_index=True)
    icao = icao.drop_duplicates(
        subset=["Frequency", "Latitude", "Longitude"], keep="last", ignore_index=True
    )
    icao = icao.astype(
        {
            "Frequency": "float64",
            "Latitude": "float32",
            "Longitude": "float32",
            "Description": "string",
        }
    )
    # TODO: Eliminate this eventually
    icao.loc[np.isclose(icao.Longitude, -472.033447), "Longitude"] = -47.2033447
    icao.loc[np.isclose(icao.Longitude, 69.934998), "Longitude"] = -69.934998

    return _save_df(icao, folder, "aero")

## Base Consolidada ANATEL

In [ ]:
def validar_coords(
    row: pd.Series,  # Linha de um DataFrame
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> List[str]:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativ do IBGE"""

    mun, cod, lat, long = (
        row.Município,
        row.Código_Município,
        row.Latitude,
        row.Longitude,
    )
    is_valid = "-1"
    conn = connect_db() if connector is None else connector
    crsr = conn.cursor()
    sql = SQL_VALIDA_COORD.format(long, lat, cod)
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is not None:
        mun = result.NO_MUNICIPIO
        lat = result.NU_LATITUDE
        long = result.NU_LONGITUDE
        is_valid = result.COORD_VALIDA
    if connector is None:
        del conn
    return [str(mun), str(lat), str(long), str(is_valid)]

In [ ]:
def update_cached_df(df: pd.DataFrame, df_cache: pd.DataFrame) -> pd.DataFrame:
    """Mescla ambos dataframes eliminando os excluídos (existentes somente em df_cache)"""

    # Merge dataframes based on all columns except "Coords_Valida_IBGE"
    merged = pd.merge(
        df_cache,
        df,
        on=list(df.columns),
        how="outer",
        indicator=True,
        copy=False,
        validate="one_to_one",
    ).astype("string")

    # Exclude rows only present in df_cache
    df_cache = merged[merged["_merge"] != "left_only"]

    # inplace=True not working
    df_cache.loc[:, ["Latitude", "Longitude"]] = df_cache.loc[
        :, ["Latitude", "Longitude"]
    ].fillna("-1")

    # # Drop the _merge column
    return df_cache.drop(columns="_merge")

In [ ]:
def _validar_coords_base(
    df: pd.DataFrame,  # DataFrame com os dados da Anatel
    df_cache: pd.DataFrame,  # DataFrame validado anteriormente, usado como cache
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
    """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

    df_cache = update_cached_df(df.astype("string"), df_cache.astype("string"))

    municipios = pd.read_csv(
        Path(__file__).parent / "arquivos" / "municipios_coordenadas.csv",
        usecols=["codigo_ibge"],
        dtype="string",
    )

    # valida os códigos de municípios
    valid_cod_mun = df_cache.Código_Município.isin(municipios.codigo_ibge)

    df_cache.loc[~valid_cod_mun, "Coords_Valida_IBGE"] = "-1"

    subset = df_cache.Coords_Valida_IBGE.isna()

    if linhas := list(
        df_cache.loc[
            subset, ["Município", "Código_Município", "Latitude", "Longitude"]
        ].itertuples()
    ):
        func = partialler(validar_coords, connector=connector)

        # Gambiarra para evitar compartilhamento da mesma conexão de banco em diferentes threads
        n_workers = 1 if connector is not None else 20

        ibge = [
            "Município_IBGE",
            "Latitude_IBGE",
            "Longitude_IBGE",
            "Coords_Valida_IBGE",
        ]

        df_cache.loc[subset, ibge] = parallel(
            func, linhas, threadpool=True, n_workers=n_workers, progress=True
        )

    df_cache = df_cache.astype("string")

    df_cache.loc[df_cache.Coords_Valida_IBGE == "-1", "Coords_Valida_IBGE"] = pd.NA

    return df_cache

In [ ]:
def update_base(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    clientMongoDB: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
    conn_threads: bool = False,  # Flag para criar uma conexão de banco por thread
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder)
    radcom = update_radcom(conn, folder)
    mosaico = update_srd(clientMongoDB, folder)
    telecom = update_telecom(clientMongoDB, folder)

    df = (
        pd.concat([mosaico, radcom, stel, telecom])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    ).astype("string")

    # inplace not working!
    df.loc[:, ["Latitude", "Longitude"]] = df.loc[:, ["Latitude", "Longitude"]].fillna(
        "0"
    )
    try:
        df_cache = _read_df(folder, "base")
    except FileNotFoundError:
        df_cache = pd.DataFrame(columns=df.columns.to_list() + ["Coords_Valida_IBGE"])

    connector = None if conn_threads else conn

    df_cache = _validar_coords_base(df, df_cache, connector)

    return _save_df(df_cache, folder, "base")